In [2]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

# import warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning);

import sys
sys.executable

'/home/mingrui/anaconda3/envs/py36_cu90/bin/python'

In [3]:
import os
import glob
import shutil
from random import shuffle

# plotting
import PIL
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from IPython.display import Image
plt.rcParams['figure.figsize'] = [15, 15]

import deephistopath.wsi.slide as slide

import py_wsi
import py_wsi.imagepy_toolkit as tk

from wsi_utils import multiprocessing_pipeline, file_stats, turtle_stats

import pandas as pd
pd.set_option('display.max_columns',None)

# Setup multi classes

In [4]:
tumor_folder_names = [['TCGA_Glioma'], ['Meningiomas', 'CGGA_Meningioma'], ['TCGA_Lung'], ['TCGA_Kidney'], 
                      ['TCGA_Breast'], ['TCGA_Uterus'], ['TCGA_Colorectal'], 
                      ['Ependymoma_Craniopharyngioma', 'CGGA_EC'], ['TCGA_WSI_glioma', 'TCGA_Glioma_Frozen']]
tumor_type_list = []

SYNOLOGY_DIR = '/media/pathimg/'
PATH_NET_DIR = '/media/disk2/PathologyNet'
#PATH_NET_DIR = './data/'

for tumor in tumor_folder_names:
    # use first as origin folder name
    tumor_dir = os.path.join(SYNOLOGY_DIR, tumor[0])
    # use last as destination folder name
    db_dir = os.path.join(PATH_NET_DIR, tumor[-1])
    # name db with destination folder name
    db_name = tumor[-1] + '_patch_db'
    tumor_type_list.append([tumor_dir, db_dir, db_name])
    
print(tumor_type_list)

[['/media/pathimg/TCGA_Glioma', '/media/disk2/PathologyNet/TCGA_Glioma', 'TCGA_Glioma_patch_db'], ['/media/pathimg/Meningiomas', '/media/disk2/PathologyNet/CGGA_Meningioma', 'CGGA_Meningioma_patch_db'], ['/media/pathimg/TCGA_Lung', '/media/disk2/PathologyNet/TCGA_Lung', 'TCGA_Lung_patch_db'], ['/media/pathimg/TCGA_Kidney', '/media/disk2/PathologyNet/TCGA_Kidney', 'TCGA_Kidney_patch_db'], ['/media/pathimg/TCGA_Breast', '/media/disk2/PathologyNet/TCGA_Breast', 'TCGA_Breast_patch_db'], ['/media/pathimg/TCGA_Uterus', '/media/disk2/PathologyNet/TCGA_Uterus', 'TCGA_Uterus_patch_db'], ['/media/pathimg/TCGA_Colorectal', '/media/disk2/PathologyNet/TCGA_Colorectal', 'TCGA_Colorectal_patch_db'], ['/media/pathimg/TCGA_WSI_glioma', '/media/disk2/PathologyNet/TCGA_Glioma_Frozen', 'TCGA_Glioma_Frozen_patch_db'], ['/media/pathimg/Ependymoma_Craniopharyngioma', '/media/disk2/PathologyNet/CGGA_EC', 'CGGA_EC_patch_db']]


In [5]:
def process_tumor_type(tumor_dir, db_dir, db_name, num_to_process):
    turtle = py_wsi.Turtle(tumor_dir, db_dir, db_name)
    turtle_stats(turtle)
#     file_stats(turtle.files_path[0])
    files_path = turtle.files_path[:num_to_process]
    files_name = turtle.files[:num_to_process]
    
    return files_path, files_name

In [7]:
%%capture
for tumor_type in tumor_type_list:
    files_path, files_name = process_tumor_type(tumor_type[0], tumor_type[1], tumor_type[2], 100)
    db_dir = tumor_type[1]
    # create db dir if doesn't exist at destination
    if os.path.exists(db_dir):
        shutil.rmtree(db_dir)
        os.mkdir(db_dir)
    else:
        os.mkdir(db_dir)
    multiprocessing_pipeline(tumor_type[0], db_dir, files_path, files_name, tile_size=1024, zoom_level=0, num_tiles=50)

# Move files to training folders

In [11]:
for tumor_type in tumor_type_list:
    tumor_name = tumor_type[2][:-9]
    tumor_db_dir = tumor_type[1]

    tumor_tile_dir = os.path.join(tumor_db_dir, 'tiles_png')

    CLASSIFICATION_DATA_DIR = './data/multiclass_large_data'
    train_dir = os.path.join(CLASSIFICATION_DATA_DIR, 'train')
    val_dir = os.path.join(CLASSIFICATION_DATA_DIR, 'val')

    train_tumor_dir = os.path.join(train_dir, tumor_name)
    val_tumor_dir = os.path.join(val_dir, tumor_name)

    if not os.path.exists(train_tumor_dir):
        os.mkdir(train_tumor_dir)

    if not os.path.exists(val_tumor_dir):
        os.mkdir(val_tumor_dir)

    all_tumor_png = glob.glob(os.path.join(tumor_tile_dir, '*', '*'), recursive=True)
    
    print(tumor_name, len(all_tumor_png))
    
    shuffle(all_tumor_png)
    train_ratio = int(len(all_tumor_png)*0.8)
    
    for img in all_tumor_png[:train_ratio]:
        shutil.move(img, train_tumor_dir)
    for img in all_tumor_png[train_ratio:]:
        shutil.move(img, val_tumor_dir)

TCGA_Glioma 5000
CGGA_Meningioma 5000
TCGA_Lung 5000
TCGA_Kidney 5000
TCGA_Breast 5000
TCGA_Uterus 5000
TCGA_Colorectal 5000
TCGA_Glioma_Frozen 4978
CGGA_EC 5000
